# Acquiring data

In [2]:
!pip install kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"anasnafisalmustofa","key":"bc42992b79b2f13675109489719558b2"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d vbookshelf/respiratory-sound-database

100% 3.68G/3.69G [00:46<00:00, 62.0MB/s]
100% 3.69G/3.69G [00:46<00:00, 85.8MB/s]


In [6]:
!ls

kaggle.json  respiratory-sound-database.zip  sample_data


In [7]:
import zipfile
zip_ref = zipfile.ZipFile('respiratory-sound-database.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()


Buat dataframe database suara paru

In [10]:
import wave
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow

%matplotlib inline

In [9]:
df_no_diagnosis = pd.read_csv('/content/files/demographic_info.txt', names = 
                 ['Patient number', 'Age', 'Sex' , 'Adult BMI (kg/m2)', 'Child Weight (kg)' , 'Child Height (cm)'], delimiter = ' ')

diagnosis = pd.read_csv('/content/files/respiratory_sound_database/Respiratory_Sound_Database/patient_diagnosis.csv', names = ['Patient number', 'Diagnosis'])

In [11]:
df =  df_no_diagnosis.join(diagnosis.set_index('Patient number'), on = 'Patient number', how = 'left')
df['Diagnosis'].value_counts()

COPD              64
Healthy           26
URTI              14
Bronchiectasis     7
Pneumonia          6
Bronchiolitis      6
LRTI               2
Asthma             1
Name: Diagnosis, dtype: int64

In [20]:
df

,Patient number,Age,Sex,Adult BMI (kg/m2),Child Weight (kg),Child Height (cm),Diagnosis
0,101,3.00,F,NaN,19.0,99.0,URTI
1,102,0.75,F,NaN,9.8,73.0,Healthy
2,103,70.00,F,33.00,NaN,NaN,Asthma
3,104,70.00,F,28.47,NaN,NaN,COPD
4,105,7.00,F,NaN,32.0,135.0,URTI
...,...,...,...,...,...,...,...
121,222,60.00,M,NaN,NaN,NaN,COPD
122,223,NaN,NaN,NaN,NaN,NaN,COPD
123,224,10.00,F,NaN,32.3,143.0,Healthy
124,225,0.83,M,NaN,7.8,74.0,Healthy


In [12]:
root = '/content/files/respiratory_sound_database/Respiratory_Sound_Database/audio_and_txt_files/'
filenames = [s.split('.')[0] for s in os.listdir(path = root) if '.txt' in s]

In [13]:
def Extract_Annotation_Data(file_name, root):
    tokens = file_name.split('_')
    recording_info = pd.DataFrame(data = [tokens], columns = ['Patient number', 'Recording index', 'Chest location','Acquisition mode','Recording equipment'])
    recording_annotations = pd.read_csv(os.path.join(root, file_name + '.txt'), names = ['Start', 'End', 'Crackles', 'Wheezes'], delimiter= '\t')
    return (recording_info, recording_annotations)

In [15]:
i_list = []
rec_annotations = []
rec_annotations_dict = {}
for s in filenames:
    (i,a) = Extract_Annotation_Data(s, root)
    i_list.append(i)
    rec_annotations.append(a)
    rec_annotations_dict[s] = a
recording_info = pd.concat(i_list, axis = 0)
recording_info.head()

,Patient number,Recording index,Chest location,Acquisition mode,Recording equipment
0,176,1b4,Al,mc,AKGC417L
0,221,2b2,Ar,mc,LittC2SE
0,175,1b1,Pr,sc,Litt3200
0,218,1b1,Pr,sc,Meditron
0,207,3b2,Pl,mc,AKGC417L


In [16]:
no_label_list = []
crack_list = []
wheeze_list = []
both_sym_list = []
filename_list = []
for f in filenames:
    d = rec_annotations_dict[f]
    no_labels = len(d[(d['Crackles'] == 0) & (d['Wheezes'] == 0)].index)
    n_crackles = len(d[(d['Crackles'] == 1) & (d['Wheezes'] == 0)].index)
    n_wheezes = len(d[(d['Crackles'] == 0) & (d['Wheezes'] == 1)].index)
    both_sym = len(d[(d['Crackles'] == 1) & (d['Wheezes'] == 1)].index)
    no_label_list.append(no_labels)
    crack_list.append(n_crackles)
    wheeze_list.append(n_wheezes)
    both_sym_list.append(both_sym)
    filename_list.append(f)

In [19]:
file_label_df = pd.DataFrame(data = {'filename':filename_list, 'no label':no_label_list, 'crackles only':crack_list, 'wheezes only':wheeze_list, 'crackles and wheezees':both_sym_list})
file_label_df

,filename,no label,crackles only,wheezes only,crackles and wheezees
0,176_1b4_Al_mc_AKGC417L,1,5,0,0
1,221_2b2_Ar_mc_LittC2SE,1,0,1,4
2,175_1b1_Pr_sc_Litt3200,1,0,3,0
3,218_1b1_Pr_sc_Meditron,2,3,2,11
4,207_3b2_Pl_mc_AKGC417L,9,0,0,0
...,...,...,...,...,...
915,218_1b1_Al_sc_Meditron,5,4,1,5
916,146_2b4_Lr_mc_AKGC417L,6,0,0,0
917,216_1b1_Pl_sc_Meditron,2,0,8,0
918,151_3p2_Pl_mc_AKGC417L,0,5,0,1


# Exploratory Data Analysis 

# Preprocessing

# Training and Testing

Metric : Accuracy, waktu komputasi prediksi 

# Integration with mobile